# <font color="green">CVC Project</font>

## <font color ="Gray"><strong><u>Team members:</u></strong></font><font color="SlateBlue"><br><ol><li>Mostafa Alaa<li>Mina Hany<li>Micheal Hesham</ol></font>

### Import needed modules

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pyzbar.pyzbar as pyzbar
import cv2
import numpy as np

%matplotlib inline

from math import pow, sqrt, atan, pi

## <font color="red">* Project starts here *</font>

In [ ]:
cap = cv2.VideoCapture(0)

while(True):
    ret, frame = cap.read()
    
    plt.imshow(frame)
    
    # find the barcodes in the image and decode each of the barcodes
    barcode = pyzbar.decode(frame)
    
    if len(barcode) > 0:
        print(barcode)
        cap.release()
        break

In [ ]:
print(len(barcode))

# extract the bounding box location of the barcode and crop the image
(x, y, w, h) = barcode[0].rect
img_gray = frame[y:y+w, x:x+h]
    
plt.imshow(img_gray, cmap="gray")
print(barcode)

In [ ]:
# Convert to gray
img_gray = cv2.cvtColor(img_gray, cv2.COLOR_BGR2GRAY)

# Threshold
img_gray = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, \
            cv2.THRESH_BINARY,11,2)

# Flip image
#img_gray = cv2.flip(img_gray, 1)

plt.imshow(img_gray, cmap="gray")

In [ ]:
# Get labels and stats
num_labels, labels_im, stats, centroids = cv2.connectedComponentsWithStats(img_gray)

print(stats)

In [ ]:
# Squares area threshold
max_area = 1250
min_area = 1000

In [ ]:
# Create copy of stats and image
image = np.zeros(img_gray.shape, dtype="uint8")
stats_cp = np.copy(stats)

# List to store each square's center point
squares = list()
    
# Loop on each label to draw a rectangle on it
for j in range(len(stats)):
        
    # Skip if it is background or if small or big region
    if stats[j, cv2.CC_STAT_AREA] < min_area or stats[j, cv2.CC_STAT_AREA] > max_area:
        continue
    
    # Get square center point
    x = int(stats[j, cv2.CC_STAT_LEFT] + (stats[j, cv2.CC_STAT_WIDTH] / 2))
    y = int(stats[j, cv2.CC_STAT_TOP] + (stats[j, cv2.CC_STAT_HEIGHT] / 2))
    # Add the square center point to the list
    squares.append((x, y))
        
    # Get the points of the small square
    pts = np.where(labels_im == j)
    # draw it
    image[pts] = 255

plt.imshow(image, cmap="gray")

In [ ]:
print(squares)

In [ ]:
lines = np.zeros(img_gray.shape, dtype="uint8")

lines = cv2.line(lines, squares[0], squares[1], (255, 255, 255), 1) 
lines = cv2.line(lines, squares[0], squares[2], (255, 255, 255), 1) 
lines = cv2.line(lines, squares[1], squares[2], (255, 255, 255), 1) 


plt.imshow(lines, cmap="gray")

In [ ]:
# List to store line point and its length
line_len = list()

# calculate the length of the lines
for i in range(len(squares)):
    x1, y1 = squares[i]

    for j in range(i+1, len(squares)):
        x2, y2 = squares[j]
        
        # Calculate length
        line = int(sqrt(pow(x2-x1, 2) + pow(y2-y1, 2)))
        
        line_len.append([(x1, y1), (x2, y2), (line)])
        
print(line_len)

In [ ]:
# Find longest line
max_line = line_len[0]

if max_line[2] < line_len[1][2] or max_line[2] < line_len[2][2]:
    if line_len[1][2] < line_len[2][2]:
        max_line = line_len[2]
    else:
        max_line = line_len[1]
        
print(max_line)

x1, y1 = max_line[0]
x2, y2 = max_line[1]

# Find coordinates of the unused square
i1 = squares.index(max_line[0])
i2 = squares.index(max_line[1])
i3 = 0

for i in range(len(squares)):
    if i != i1 and i != i2:
        i3 = i
        break
                
x3, y3 = squares[i3]

# Midpoint of longest line
ym = int(round((y2 + y1) / 2))
xm = int(round((x2 + x1) / 2))

print(x1, y1)
print(x2, y2)
print(x3, y3)
print(xm, ym)

In [ ]:
# Calculate slope
# If longest line is horizontal, angle = 135 or 315
if (y1-y2) == 0:
    if y3 > ym:
        angle = 315
    else:
        angle = 135
    
# If longest line is vertical, angle = 45 or 225
elif (x2-x1) ==0:
    if x3 > xm:
        angle = 45        
    else:
        angle = 225
    
else:
    m = (x1-x2)/(y2-y1)
    angle = int(round(atan(m) * 180 / pi))
    
    if x3 > xm:
        angle += (180-45)
    else:
        angle -= 45
        
    angle = (angle + 360) % 360
    
print(angle)
print(barcode[0].data)